# 1.Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from PIL import Image
import pytesseract
import requests
from io import BytesIO
import re

# Reading Data

In [2]:
file_dir = r'/Users/rupinajay/Developer/Amazon ML Challenge 24/dataset'

In [3]:
def get_data(dataset_name):
    file_name = f"{dataset_name}.csv"
    file_path = os.path.join(file_dir,file_name)
    
    return pd.read_csv(file_path)

In [4]:
train = get_data('train')

In [5]:
train.head()

,image_link,group_id,entity_name,entity_value
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram
4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,item_weight,1400 milligram


In [6]:
train.tail()

,image_link,group_id,entity_name,entity_value
263854,https://m.media-amazon.com/images/I/612J1R1xHl...,558806,height,5.0 centimetre
263855,https://m.media-amazon.com/images/I/61Blzh2+28...,470067,height,8.5 inch
263856,https://m.media-amazon.com/images/I/51MsegDL9V...,204245,height,43.2 centimetre
263857,https://m.media-amazon.com/images/I/510KhVw4VS...,752266,height,9.1 centimetre
263858,https://m.media-amazon.com/images/I/51lzTNLQ-6...,416664,height,27.5 centimetre


In [7]:
train.iloc[55]['image_link']

'https://m.media-amazon.com/images/I/71E7CU55dcL.jpg'

## Running Tesseract code

### Reading Image data without rotation

In [8]:
# Load the image
 
image_url = 'https://m.media-amazon.com/images/I/71E7CU55dcL.jpg'

# Fetch the image from the URL
response = requests.get(image_url)

# Open the image from the response content
img = Image.open(BytesIO(response.content))

# Extract text from the image
text = pytesseract.image_to_string(img)

# Print the extracted text
print(text)


Specifications:
Material: - borosilicate glass

Colors: ¢ 3- Natural, Pink, Gold

Weight: * 160g
Volume: ¢ 180 ml

Shape: ¢ Shiba Inu

9,5cm




### Reading Image Data with Rotation of image

In [9]:
def get_image_data(image_url):
    
    # Fetch the image from the URL
    response = requests.get(image_url)
    
    # Open the image from the response content
    img = Image.open(BytesIO(response.content))
    
    # List to store the extracted text at different rotations
    extracted_texts = []
    
    # Rotate the image in all four directions and extract text
    for angle in [0, 90, 180, 270]:
        # Rotate the image
        rotated_img = img.rotate(angle, expand=True)
        
        # Extract text from the rotated image
        text = pytesseract.image_to_string(rotated_img)
        
        # Store the extracted text in the list
        extracted_texts.append(text)
    
    return extracted_texts
    
    # The list 'extracted_texts' will now contain the extracted text at each rotation angle.


In [10]:
get_image_data('https://m.media-amazon.com/images/I/71E7CU55dcL.jpg')

['Specifications:\n\nMaterial: - borosilicate glass\n\nColors: ¢ 3- Natural, Pink, Gold\nWeight: * 160g\nVolume: - 180 ml\n\nShape: ¢ Shiba Inu\n\n9,5cm\n\n',
 '',
 'wd G‘6\n\nnu] eqrys - sadeys\nTWoOSt. s<sUMIOA\nSo9r- 34310\nPICD ‘Hurd ‘TernzeN - € « tsa0]OD\nSSU[Z aqVoTIso10g »  seLJOREL\n\nSSUOTJBDIZINIdS\n\n',
 'Specifications:\n\nMaterial: - borosilicate glass\n\nColors: ¢ 3- Natural, Pink, Gold\nWeight:\n\nVolume:\n\nShape:\n\n9,5cm\n\n']

## Cleaning Extracted Text after OCR

In [12]:
# Load the image
image_url = 'https://m.media-amazon.com/images/I/613thsN41vL.jpg'

# Fetch the image from the URL
response = requests.get(image_url)

# Open the image from the response content
img = Image.open(BytesIO(response.content))

# Extract text using pytesseract
text = pytesseract.image_to_string(img)

# Clean the extracted text
def clean_text(text):
    text = re.sub(r'[^\w\s\.\-,:]', '', text)  # Remove unwanted symbols
    text = re.sub(r'\n+', '\n', text).strip()  # Remove extra newlines
    return text

cleaned_text = clean_text(text)

# Provide the information as needed
user_info = f"""
Product Specifications:

{cleaned_text}
"""

print(user_info) 



Product Specifications:

F
BATTERY FREE
AUTOMATIC
START
27KG IN  - y, SOLID ZINC
RESISTANCE ROTOR



In [13]:
def clean_text(text):
    text = re.sub(r'[^\w\s\.\-,:\n]', '', text)  # Remove unwanted symbols but keep key-value symbols like ":" and "-"
    text = re.sub(r'\n+', '\n', text).strip()  # Remove extra newlines
    return text

In [14]:
def extract_key_value_pairs(cleaned_text):
    # Regex to capture lines like "Key: Value" or "Key - Value"
    pattern = re.compile(r'([A-Za-z\s]+)[:\-]\s*(.*)')
    key_value_pairs = {}
    
    # Split text into lines and search for key-value pairs
    for line in cleaned_text.split('\n'):
        match = pattern.match(line)
        if match:
            key = match.group(1).strip()
            value = match.group(2).strip()
            key_value_pairs[key] = value
    
    return key_value_pairs

In [15]:
txt = get_image_data('https://m.media-amazon.com/images/I/71E7CU55dcL.jpg')
cleaned_text = extract_key_value_pairs(clean_text(txt[0]))

In [16]:
from PIL import Image
import pytesseract
import re
from sklearn.feature_extraction.text import TfidfVectorizer

# Load the image
image_url = 'https://m.media-amazon.com/images/I/71E7CU55dcL.jpg'

# Fetch the image from the URL
response = requests.get(image_url)

# Open the image from the response content
img = Image.open(BytesIO(response.content))

# Extract text using pytesseract
text = pytesseract.image_to_string(img)

# General text cleaning function
def clean_text(text):
    text = re.sub(r'[^\w\s\.\-,:\n]', '', text)  # Remove unwanted symbols but keep key-value symbols like ":" and "-"
    text = re.sub(r'\n+', '\n', text).strip()  # Remove extra newlines
    return text

# Generalized function to extract key-value pairs
def extract_key_value_pairs(cleaned_text):
    # Regex to capture lines like "Key: Value" or "Key - Value"
    pattern = re.compile(r'([A-Za-z\s]+)[:\-]\s*(.*)')
    key_value_pairs = {}
    
    # Split text into lines and search for key-value pairs
    for line in cleaned_text.split('\n'):
        match = pattern.match(line)
        if match:
            key = match.group(1).strip()
            value = match.group(2).strip()
            key_value_pairs[key] = value
    
    return key_value_pairs

# Clean the text
cleaned_text = clean_text(text)
print("Cleaned Text:")
print(cleaned_text)

# Extract key-value pairs dynamically
key_value_pairs = extract_key_value_pairs(cleaned_text)
print("\nExtracted Key-Value Pairs:")
print(key_value_pairs)

# You can now use these key-value pairs as you wish
# For example, convert to TF-IDF features or feed into a model

# If needed, convert key-value pairs back into a formatted string for use
formatted_string = "\n".join([f"{key}: {value}" for key, value in key_value_pairs.items()])
print("\nFormatted Output:")
print(formatted_string)

# Example: Convert cleaned text into TF-IDF features
texts = [cleaned_text]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(texts)

# Now X is ready for use in a prediction model


Cleaned Text:
Specifications:
Material: - borosilicate glass
Colors:  3- Natural, Pink, Gold
Weight:  160g
Volume:  180 ml
Shape:  Shiba Inu
9,5cm

Extracted Key-Value Pairs:
{'Specifications': '', 'Material': '- borosilicate glass', 'Colors': '3- Natural, Pink, Gold', 'Weight': '160g', 'Volume': '180 ml', 'Shape': 'Shiba Inu'}

Formatted Output:
Specifications: 
Material: - borosilicate glass
Colors: 3- Natural, Pink, Gold
Weight: 160g
Volume: 180 ml
Shape: Shiba Inu
